In [ ]:
!pip install pymupdf

In [ ]:
import fitz  # PyMuPDF
import re
import pandas as pd

# Ingesta de Datos de Boletines

In [ ]:
import os
import requests

In [ ]:
# Instalar gdown para descargar desde Google Drive
!pip install -q gdown

# Descargar el archivo ZIP desde Google Drive (ID del archivo)
!gdown --id 1VsKDt8KTn7_n_6slX6vYEaOTkloS9UqP --output boletines.zip

In [ ]:
# Descomprimir el archivo
import zipfile
import os

In [ ]:
zip_path = "boletines.zip"
extract_folder = "boletines_extraidos"

# Crear carpeta de salida si no existe
os.makedirs(extract_folder, exist_ok=True)

# Extraer los archivos
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Listar archivos extraídos
import os
archivos = os.listdir(extract_folder)
print("Archivos extraídos:")
for archivo in archivos:
    print(archivo)

In [ ]:
import os

carpeta = "/content/boletines_extraidos/Boletines/boletines_2024" # aqui va la carpeta en drive donde estan los documentos

pdfs = [f for f in os.listdir(carpeta) if f.endswith(".pdf")]

print("PDFs encontrados:", pdfs)

Por cada pdf o boletin se realiza la lectura y separacion de documentos.

In [ ]:
# --- 2. Procesar cada PDF ---
df_total = pd.DataFrame()

for numero in pdfs:
    pdf_path = os.path.join(carpeta, numero)
    if not os.path.exists(pdf_path):
        continue  # Saltar si no se descargó

    try:
        doc = fitz.open(pdf_path)
        texto_sumario = ""

        # Ignorar página 1 y 2
        # Ignorar página 0, 1 y la última página
        for i in range(2, len(doc) - 1):
            texto_sumario += doc[i].get_text()
        #for i in range(2, len(doc)):
        #    texto_sumario += doc[i].get_text()

        # Patrón que detecta bloques finalizados con OP
        patron_op = re.compile(r"OP\s*N[°º]:\s*[A-Z]*\d{6,}", re.IGNORECASE)
        matches = list(patron_op.finditer(texto_sumario))

        if not matches:
            documentos = [texto_sumario.strip()]
        else:
            documentos = []
            start_idx = 0
            for m in matches:
                end_idx = m.end()
                bloque = texto_sumario[start_idx:end_idx].strip()
                documentos.append(bloque)
                start_idx = end_idx
            if start_idx < len(texto_sumario):
                documentos.append(texto_sumario[start_idx:].strip())

        # Extraer OP
        def extraer_op_final(texto):
            match = re.search(r"OP\s*N[°º]:\s*([A-Z]*\d{6,})\s*$", texto.strip(), re.IGNORECASE)
            return match.group(1) if match else None

        df = pd.DataFrame({
            "Boletin_N": numero,
            "Documento_N": range(1, len(documentos)+1),
            "Texto": documentos
        })

        df["OP_Numero"] = df["Texto"].apply(extraer_op_final)
        df_total = pd.concat([df_total, df], ignore_index=True)
        print(f" Procesado boletín {numero} con {len(df)} documentos.")

    except Exception as e:
        print(f"Error procesando boletín {numero}: {e}")

In [ ]:
df_total

In [ ]:
df_total.info()

# Limpiza de los datos

Existen documentos que no estan asociados a un OP_Numero, se trata de encabezados o finales de documentos.

In [ ]:
df_total[df_total["OP_Numero"].isna()]

In [ ]:
df_total = df_total[df_total["OP_Numero"].notna()]

In [ ]:
df_total.reset_index(drop=True, inplace=True)

In [ ]:
df_total

Algunos documentos tienen pie de pagina y encabezados dentro del documento.

In [ ]:
def eliminar_pies_pagina(texto):
    lineas = texto.splitlines()
    nuevas_lineas = []
    skip = 0

    for i, linea in enumerate(lineas):
        if skip > 0:
            skip -= 1
            continue
        if re.match(r"Pág\.\s*N°\s*\d+", linea.strip()):
            skip = 3  # saltar esta línea y las 3 siguientes
            continue
        nuevas_lineas.append(linea)

    return "\n".join(nuevas_lineas)



In [ ]:
# Aplicar limpieza
df_total["Texto_Limpio"] = df_total["Texto"].apply(eliminar_pies_pagina)

In [ ]:
df_total.sample(3)

In [ ]:
df_total.loc[1]['Texto']

In [ ]:
df_total.loc[1]['Texto_Limpio']

# Modelo para predecir etiquetas


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                       model="hackathon-pln-es/bertin-roberta-base-zeroshot-esnli")

## Texto de ejemplo del modelo

In [ ]:
classifier(
    "El autor se perfila, a los 50 años de su muerte, como uno de los grandes de su siglo",
    candidate_labels=["cultura", "sociedad", "economia", "salud", "deportes"],
    hypothesis_template="Esta oración es sobre {}."
)


In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

## Clasificador de documentos

In [ ]:
class DocumentClassifier:
    """
    Clasificador automático de documentos usando BART-large-MNLI
    para clasificación zero-shot de boletines oficiales
    """

    def __init__(self, model_name: str = "facebook/bart-large-mnli"):
        """
        Inicializa el clasificador

        Args:
            model_name: Nombre del modelo de Hugging Face
        """
        print("Cargando modelo BART-large-MNLI...")
        self.classifier = pipeline(
            "zero-shot-classification",
            model=model_name,
            device=0 if torch.cuda.is_available() else -1  # GPU si está disponible
        )
        print("Modelo cargado exitosamente!")

        # Etiquetas para clasificación
        self.etiquetas_boletin = [
            "Leyes",
            "Decisiones Administrativas",
            "Resoluciones Delegadas",
            "Resoluciones Ministeriales",
            "Resoluciones (Secretaría de Obras Públicas)",
            "Licitaciones Públicas",
            "Adjudicaciones Simples",
            "Contrataciones Abreviadas",
            "Concesiones de Agua Pública",
            "Sentencias",
            "Sucesorios",
            "Edictos de Quiebras",
            "Concursos Civiles o Preventivos",
            "Edictos Judiciales",
            "Constituciones de Sociedad",
            "Asambleas Comerciales",
            "Asambleas Civiles",
            "Avisos Generales",
            "Recaudación"
        ]

    def preprocess_text(self, text: str, max_length: int = 512) -> str:
        """
        Preprocesa el texto para optimizar la clasificación

        Args:
            text: Texto a procesar
            max_length: Longitud máxima del texto

        Returns:
            Texto preprocesado
        """
        if not isinstance(text, str) or not text.strip():
            return ""

        # Limpiar texto básico
        text = text.strip()

        # Tomar principalmente el inicio del documento (más informativo)
        # y algo del final si es muy largo
        if len(text) > max_length:
            # Tomar primeros 400 caracteres y últimos 100
            text = text[:400] + "..." + text[-100:]

        return text

    def classify_single_document(self, text: str, threshold: float = 0.5) -> Dict:
        """
        Clasifica un solo documento

        Args:
            text: Texto del documento
            threshold: Umbral mínimo de confianza

        Returns:
            Diccionario con resultado de clasificación
        """
        # Preprocesar texto
        processed_text = self.preprocess_text(text)

        if not processed_text:
            return {
                'etiqueta_predicha': 'Sin clasificar',
                'confianza': 0.0,
                'top_3_etiquetas': [],
                'scores_completos': {}
            }

        try:
            # Realizar clasificación
            resultado = self.classifier(
                processed_text,
                self.etiquetas_boletin,
                multi_label=False
            )

            # Extraer resultados
            etiqueta_principal = resultado['labels'][0]
            confianza_principal = resultado['scores'][0]

            # Top 3 etiquetas con scores
            top_3 = [
                {
                    'etiqueta': resultado['labels'][i],
                    'score': resultado['scores'][i]
                }
                for i in range(min(3, len(resultado['labels'])))
            ]

            # Scores completos
            scores_completos = dict(zip(resultado['labels'], resultado['scores']))

            # Aplicar umbral de confianza
            if confianza_principal < threshold:
                etiqueta_final = 'Clasificación incierta'
            else:
                etiqueta_final = etiqueta_principal

            return {
                'etiqueta_predicha': etiqueta_final,
                'confianza': confianza_principal,
                'top_3_etiquetas': top_3,
                'scores_completos': scores_completos
            }

        except Exception as e:
            print(f"Error en clasificación: {str(e)}")
            return {
                'etiqueta_predicha': 'Error en clasificación',
                'confianza': 0.0,
                'top_3_etiquetas': [],
                'scores_completos': {}
            }

    def classify_dataframe(self, df: pd.DataFrame, text_column: str = 'Texto',
                          threshold: float = 0.5, batch_size: int = 10) -> pd.DataFrame:
        """
        Clasifica todos los documentos en un DataFrame

        Args:
            df: DataFrame con los documentos
            text_column: Nombre de la columna con el texto
            threshold: Umbral de confianza
            batch_size: Tamaño de lote para procesamiento

        Returns:
            DataFrame con las clasificaciones agregadas
        """
        print(f"Clasificando {len(df)} documentos...")

        # Copiar DataFrame para no modificar el original
        df_resultado = df.copy()

        # Listas para almacenar resultados
        etiquetas_predichas = []
        confianzas = []
        top_3_lists = []

        # Procesar en lotes para mostrar progreso
        for i in range(0, len(df), batch_size):
            batch_end = min(i + batch_size, len(df))
            print(f"Procesando documentos {i+1}-{batch_end} de {len(df)}")

            # Procesar cada documento en el lote
            for idx in range(i, batch_end):
                texto = df.iloc[idx][text_column]
                resultado = self.classify_single_document(texto, threshold)

                etiquetas_predichas.append(resultado['etiqueta_predicha'])
                confianzas.append(resultado['confianza'])
                top_3_lists.append(resultado['top_3_etiquetas'])

        # Agregar resultados al DataFrame
        df_resultado['Etiqueta_Predicha'] = etiquetas_predichas
        df_resultado['Confianza'] = confianzas
        df_resultado['Top_3_Etiquetas'] = top_3_lists

        print("Clasificación completada!")
        return df_resultado

    def get_classification_summary(self, df_classified: pd.DataFrame) -> pd.DataFrame:
        """
        Genera un resumen de las clasificaciones

        Args:
            df_classified: DataFrame con clasificaciones

        Returns:
            DataFrame con resumen estadístico
        """
        summary = df_classified.groupby('Etiqueta_Predicha').agg({
            'Confianza': ['count', 'mean', 'std', 'min', 'max']
        }).round(3)

        summary.columns = ['Cantidad', 'Confianza_Media', 'Confianza_Std', 'Confianza_Min', 'Confianza_Max']
        summary = summary.reset_index()
        summary = summary.sort_values('Cantidad', ascending=False)

        return summary

    def analyze_low_confidence_predictions(self, df_classified: pd.DataFrame,
                                         threshold: float = 0.7) -> pd.DataFrame:
        """
        Analiza las predicciones con baja confianza para revisión manual

        Args:
            df_classified: DataFrame con clasificaciones
            threshold: Umbral para considerar baja confianza

        Returns:
            DataFrame con documentos de baja confianza
        """
        low_confidence = df_classified[df_classified['Confianza'] < threshold].copy()

        if len(low_confidence) > 0:
            print(f"Encontrados {len(low_confidence)} documentos con confianza < {threshold}")
            print("Estos documentos podrían requerir revisión manual.")

        return low_confidence.sort_values('Confianza')

Instanciamos el clasificador

In [ ]:
clasificador = DocumentClassifier()

In [ ]:
df_total

In [ ]:
resultado_individual = clasificador.classify_single_document(df_total.iloc[0]['Texto_Limpio'])

In [ ]:
print(f" Documento: {df_total.iloc[0]['Texto_Limpio']}...")
print(f"  Etiqueta predicha: {resultado_individual['etiqueta_predicha']}")
print(f" Confianza: {resultado_individual['confianza']:.3f}")
print(" Top 3 etiquetas:")
for i, item in enumerate(resultado_individual['top_3_etiquetas'][:3]):
    print(f"   {i+1}. {item['etiqueta']}: {item['score']:.3f}")

## Clasificador Masivo

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional
import torch
from transformers import pipeline
from tqdm import tqdm
import time
import gc

In [ ]:
from google.colab import files

In [ ]:
import shutil

In [ ]:
class DocumentClassifierOptimized:
    """
    Versión optimizada del clasificador para procesar grandes volúmenes de documentos
    """

    def __init__(self, model_name: str = "facebook/bart-large-mnli"):
        """
        Inicializa el clasificador optimizado
        """
        print("Cargando modelo BART-large-MNLI...")

        # Configuración optimizada del pipeline
        self.classifier = pipeline(
            "zero-shot-classification",
            model=model_name,
            device=0 if torch.cuda.is_available() else -1,
            # Optimizaciones de memoria
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            model_kwargs={
                "low_cpu_mem_usage": True,
                "use_cache": False  # Reduce memoria
            }
        )

        print(f"Modelo cargado en: {'GPU' if torch.cuda.is_available() else 'CPU'}")

        # Etiquetas para clasificación
        self.etiquetas_boletin = [
            "Leyes",
            "Decisiones Administrativas",
            "Resoluciones Delegadas",
            "Resoluciones Ministeriales",
            "Resoluciones (Secretaría de Obras Públicas)",
            "Licitaciones Públicas",
            "Adjudicaciones Simples",
            "Contrataciones Abreviadas",
            "Concesiones de Agua Pública",
            "Sentencias",
            "Sucesorios",
            "Edictos de Quiebras",
            "Concursos Civiles o Preventivos",
            "Edictos Judiciales",
            "Constituciones de Sociedad",
            "Asambleas Comerciales",
            "Asambleas Civiles",
            "Avisos Generales",
            "Recaudación"
        ]

    def preprocess_batch_texts(self, texts: List[str], max_length: int = 400) -> List[str]:
        """
        Preprocesa un lote de textos de manera eficiente
        """
        processed_texts = []

        for text in texts:
            if not isinstance(text, str) or not text.strip():
                processed_texts.append("")
                continue

            text = text.strip()

            # Truncar texto de manera inteligente
            if len(text) > max_length:
                # Tomar inicio y final del texto
                text = text[:int(max_length*0.8)] + "..." + text[-int(max_length*0.2):]

            processed_texts.append(text)

        return processed_texts

    def classify_batch(self, texts: List[str]) -> List[Dict]:
        """
        Clasifica un lote de textos de manera eficiente
        """
        # Preprocesar lote
        processed_texts = self.preprocess_batch_texts(texts)

        results = []

        for text in processed_texts:
            if not text:
                # Resultado vacío para textos sin contenido
                empty_result = {etiqueta: 0.0 for etiqueta in self.etiquetas_boletin}
                results.append(empty_result)
                continue

            try:
                # Clasificar texto individual
                resultado = self.classifier(
                    text,
                    self.etiquetas_boletin,
                    multi_label=False
                )

                # Convertir a diccionario de scores
                scores_dict = dict(zip(resultado['labels'], resultado['scores']))

                # Asegurar que todas las etiquetas estén presentes
                complete_scores = {}
                for etiqueta in self.etiquetas_boletin:
                    complete_scores[etiqueta] = scores_dict.get(etiqueta, 0.0)

                results.append(complete_scores)

            except Exception as e:
                print(f"Error procesando texto: {str(e)[:100]}...")
                # Resultado con scores en 0 en caso de error
                error_result = {etiqueta: 0.0 for etiqueta in self.etiquetas_boletin}
                results.append(error_result)

        return results

    def classify_dataframe_optimized(self,
                                   df: pd.DataFrame,
                                   text_column: str,
                                   batch_size: int = 8,
                                   save_progress: bool = True,
                                   checkpoint_every: int = 1000) -> pd.DataFrame:
        """
        Clasifica DataFrame completo con optimizaciones para grandes volúmenes

        Args:
            df: DataFrame con documentos
            text_column: Nombre de columna con texto
            batch_size: Tamaño de lote (reducido para optimizar memoria)
            save_progress: Si guardar progreso periódicamente
            checkpoint_every: Cada cuántos documentos guardar checkpoint

        Returns:
            DataFrame con columnas de scores para cada etiqueta
        """
        print(f"Iniciando clasificación de {len(df)} documentos...")
        print(f"Batch size: {batch_size}")
        print(f"Etiquetas a clasificar: {len(self.etiquetas_boletin)}")

        # Copiar DataFrame
        df_resultado = df.copy()

        # Inicializar columnas de scores
        for etiqueta in self.etiquetas_boletin:
            df_resultado[f'Score_{etiqueta}'] = 0.0

        # Variables para tracking
        total_batches = (len(df) + batch_size - 1) // batch_size
        start_time = time.time()
        processed_docs = 0

        # Barra de progreso
        pbar = tqdm(total=len(df), desc="Clasificando documentos")

        try:
            # Procesar en lotes
            for batch_idx in range(0, len(df), batch_size):
                batch_end = min(batch_idx + batch_size, len(df))

                # Extraer textos del lote
                batch_texts = df.iloc[batch_idx:batch_end][text_column].tolist()

                # Clasificar lote
                batch_results = self.classify_batch(batch_texts)

                # Asignar resultados al DataFrame
                for i, scores_dict in enumerate(batch_results):
                    doc_idx = batch_idx + i
                    for etiqueta, score in scores_dict.items():
                        df_resultado.loc[doc_idx, f'Score_{etiqueta}'] = score

                # Actualizar progreso
                processed_docs += len(batch_texts)
                pbar.update(len(batch_texts))

                # Estadísticas de tiempo
                elapsed_time = time.time() - start_time
                docs_per_second = processed_docs / elapsed_time
                remaining_docs = len(df) - processed_docs
                eta_seconds = remaining_docs / docs_per_second if docs_per_second > 0 else 0

                pbar.set_postfix({
                    'Docs/s': f'{docs_per_second:.2f}',
                    'ETA': f'{eta_seconds/60:.1f}min'
                })

                # Checkpoint periódico
                if save_progress and processed_docs % checkpoint_every == 0:
                    checkpoint_file = f'classification_checkpoint_{processed_docs}.pkl'
                    df_resultado.to_pickle(checkpoint_file)
                    # Ahora lo descargás automáticamente

                    print(f"\nCheckpoint guardado: {checkpoint_file}")

                # Limpiar memoria periódicamente
                if batch_idx % (batch_size * 10) == 0:
                    gc.collect()
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()

        finally:
            pbar.close()

        # Agregar columnas de análisis
        df_resultado = self._add_analysis_columns(df_resultado)

        total_time = time.time() - start_time
        print(f"\n✅ Clasificación completada!")
        print(f"⏱️  Tiempo total: {total_time/60:.2f} minutos")
        print(f"📊 Velocidad promedio: {len(df)/total_time:.2f} docs/segundo")

        return df_resultado

    def _add_analysis_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Agrega columnas de análisis basadas en los scores
        """
        print("Agregando columnas de análisis...")

        # Columnas de scores
        score_columns = [col for col in df.columns if col.startswith('Score_')]

        # Etiqueta con mayor score
        df['Etiqueta_Predicha'] = df[score_columns].idxmax(axis=1).str.replace('Score_', '')

        # Confianza máxima
        df['Confianza_Maxima'] = df[score_columns].max(axis=1)

        # Top 3 etiquetas
        def get_top_3(row):
            scores = [(col.replace('Score_', ''), row[col]) for col in score_columns]
            scores.sort(key=lambda x: x[1], reverse=True)
            return scores[:3]

        df['Top_3_Etiquetas'] = df.apply(get_top_3, axis=1)

        # Diferencia entre top 2 (indica certeza)
        def get_confidence_gap(row):
            scores = [row[col] for col in score_columns]
            scores.sort(reverse=True)
            return scores[0] - scores[1] if len(scores) > 1 else 0

        df['Gap_Confianza'] = df.apply(get_confidence_gap, axis=1)

        return df

    def get_classification_summary(self, df_classified: pd.DataFrame) -> pd.DataFrame:
        """
        Genera resumen estadístico de las clasificaciones
        """
        print("Generando resumen de clasificación...")

        summary = df_classified.groupby('Etiqueta_Predicha').agg({
            'Confianza_Maxima': ['count', 'mean', 'std', 'min', 'max'],
            'Gap_Confianza': 'mean'
        }).round(4)

        summary.columns = ['Cantidad', 'Confianza_Media', 'Confianza_Std',
                          'Confianza_Min', 'Confianza_Max', 'Gap_Promedio']
        summary = summary.reset_index()
        summary = summary.sort_values('Cantidad', ascending=False)

        # Calcular porcentajes
        summary['Porcentaje'] = (summary['Cantidad'] / len(df_classified) * 100).round(2)

        return summary

    def save_results(self, df_classified: pd.DataFrame, filename: str = "documentos_clasificados"):
        """
        Guarda los resultados en múltiples formatos
        """
        print(f"Guardando resultados como {filename}...")

        # Guardar DataFrame completo
        df_classified.to_pickle(f"{filename}.pkl")
        df_classified.to_csv(f"{filename}.csv", index=False)

        # Guardar solo scores en archivo separado
        score_columns = [col for col in df_classified.columns if col.startswith('Score_')]
        analysis_columns = ['Etiqueta_Predicha', 'Confianza_Maxima', 'Gap_Confianza']

        df_scores = df_classified[score_columns + analysis_columns]
        df_scores.to_csv(f"{filename}_scores_only.csv", index=False)

        print("✅ Resultados guardados!")


In [ ]:
# Opción 2: Usando la clase (más control)
classifier = DocumentClassifierOptimized()
df_classified = classifier.classify_dataframe_optimized(
    df=df_total,
    text_column="Texto_Limpio",
    batch_size=8
)

In [ ]:
df_classified

In [ ]:
# Supongamos que termina tu análisis y guardás el archivo
df_classified.to_csv("resumen.csv", index=False)

# Ahora lo descargás automáticamente
files.download("resumen.csv")

# Prediccion en la Busqueda de Documentos

In [ ]:
%pip install sentence-transformers faiss-cpu pandas

In [ ]:
# Paso 1: Importar librerías
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [ ]:
import csv
import sys


file_path = "/content/resumen.csv"
# Aumentar el límite de campo para campos muy largos
csv.field_size_limit(sys.maxsize)

# Forzar lectura tolerante a errores y texto largo
df = pd.read_csv(
    file_path,
    encoding="utf-8",
    quoting=csv.QUOTE_ALL,  # requiere que todos los campos estén entre comillas
    engine="python",
    on_bad_lines="warn"  # mostrar advertencias si encuentra líneas mal formadas
)

In [ ]:
# Reemplazá por la ruta correcta de tu archivo
#df = pd.read_pickle("/content/classification_checkpoint_15000.pkl")

# Verificá los primeros registros
df

In [ ]:
df = df.dropna(subset=["Texto_Limpio"])  # aseguramos que no haya nulos
df.reset_index(drop=True, inplace=True)

# Opcional: Previsualización
df.head()

In [ ]:
df.info()

## Evaluacion de la Clasificacion

Evaluación de la Clasificación Automática – Validación por Muestreo
Para validar el rendimiento del modelo de clasificación automática sin etiquetar manualmente los 15.000 documentos, aplicamos estadística inferencial para determinar un tamaño de muestra representativo.

Parámetros definidos:
Tamaño de población (N): 15.000 documentos

Nivel de confianza: 90%

Margen de error permitido: ±10%

Proporción esperada (p): 0.5 (caso conservador)

In [ ]:
import numpy as np

# Parámetros
N = 15114           # Población
Z = 1.645           # Z-score para 90% confianza
e = 0.10            # Margen de error
p = 0.5             # Proporción esperada

# Tamaño de muestra sin corrección
n_0 = (Z**2 * p * (1 - p)) / (e**2)

# Corrección poblacional finita
n = n_0 / (1 + (n_0 - 1) / N)
n = int(np.ceil(n))

print(f"📏 Tamaño óptimo de muestra: {n} documentos")


In [ ]:
df_muestra = df.sample(n=n, random_state=42).copy()

In [ ]:
df_muestra

In [ ]:
df_muestra.to_excel("muestra_validacion_modelo.xlsx", index=False)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Reconstruir el dataset entregado por el usuario
data = {
    "Etiqueta_Predicha": [
        "Resoluciones Delegadas", "Adjudicaciones Simples", "Contrataciones Abreviadas", "Licitaciones Públicas",
        "Contrataciones Abreviadas", "Sucesorios", "Asambleas Comerciales", "Edictos de Quiebras", "Contrataciones Abreviadas",
        "Resoluciones Ministeriales", "Sucesorios", "Constituciones de Sociedad", "Concursos Civiles o Preventivos",
        "Sucesorios", "Contrataciones Abreviadas", "Resoluciones (Secretaría de Obras Públicas)", "Decisiones Administrativas",
        "Resoluciones Ministeriales", "Decisiones Administrativas", "Sucesorios", "Asambleas Comerciales", "Licitaciones Públicas",
        "Sucesorios", "Sucesorios", "Resoluciones Ministeriales", "Sentencias", "Contrataciones Abreviadas",
        "Concesiones de Agua Pública", "Resoluciones Ministeriales", "Edictos de Quiebras", "Decisiones Administrativas",
        "Decisiones Administrativas", "Decisiones Administrativas", "Sucesorios", "Sucesorios", "Sucesorios",
        "Resoluciones Ministeriales", "Resoluciones Delegadas", "Leyes", "Contrataciones Abreviadas", "Sucesorios",
        "Resoluciones Ministeriales", "Contrataciones Abreviadas", "Adjudicaciones Simples", "Sentencias",
        "Resoluciones Ministeriales", "Sucesorios", "Recaudación", "Leyes", "Adjudicaciones Simples",
        "Contrataciones Abreviadas", "Licitaciones Públicas", "Resoluciones Ministeriales", "Asambleas Comerciales",
        "Contrataciones Abreviadas", "Sucesorios", "Sucesorios", "Resoluciones Delegadas", "Contrataciones Abreviadas",
        "Sucesorios", "Edictos de Quiebras", "Sentencias", "Adjudicaciones Simples", "Contrataciones Abreviadas",
        "Licitaciones Públicas", "Decisiones Administrativas", "Licitaciones Públicas"
    ],
    "Etiqueta_Verdadera": [
        "Avisos Generales", "Adjudicaciones Simples", "Contrataciones Abreviadas", "Licitaciones Públicas",
        "Contrataciones Abreviadas", "Sucesorios", "Asambleas Comerciales", "Edictos de Quiebras", "POSESIONES VEINTEAÑALES",
        "Decretos", "Sucesorios", "Constituciones de Sociedad", "REMATES JUDICIALES", "Sucesorios", "Contrataciones Abreviadas",
        "Resoluciones (Secretaría de Obras Públicas)", "Decisiones Administrativas", "ASAMBLEAS CIVILES",
        "Decisiones Administrativas", "Sucesorios", "Avisos Comerciales", "CONVOCATORIAS A AUDIENCIA PÚBLICA",
        "Sucesorios", "Sucesorios", "Decretos", "POSESIONES VEINTEAÑALES", "Contrataciones Abreviadas",
        "Concesiones de Agua Pública", "Resoluciones Ministeriales", "Edictos de Quiebras", "NOTIFICACIONES ADMINISTRATIVAS",
        "Decisiones Administrativas", "Decisiones Administrativas", "Sucesorios", "Sucesorios", "Sucesorios",
        "Resoluciones Delegadas", "Resoluciones Delegadas", "NOTIFICACIONES ADMINISTRATIVAS", "Contrataciones Abreviadas",
        "Sucesorios", "Resoluciones Delegadas", "Contrataciones Abreviadas", "Adjudicaciones Simples", "EDICTOS DE MINAS",
        "DECRETOS", "Sucesorios", "NOTIFICACIONES ADMINISTRATIVAS", "EDICTOS DE MINAS", "Adjudicaciones Simples",
        "Contrataciones Abreviadas", "Licitaciones Públicas", "Resoluciones Ministeriales", "Asambleas Comerciales",
        "Contrataciones Abreviadas", "Sucesorios", "Sucesorios", "Resoluciones Delegadas", "Contrataciones Abreviadas",
        "Sucesorios", "Edictos de Quiebras", "NOTIFICACIONES ADMINISTRATIVAS", "Adjudicaciones Simples",
        "Contrataciones Abreviadas", "Licitaciones Públicas", "Decisiones Administrativas", "Licitaciones Públicas"
    ]
}

In [ ]:
# Crear DataFrame
df_eval = pd.DataFrame(data)

In [ ]:
df_eval

In [ ]:
# Identificar etiquetas verdaderas no vistas por el modelo
etiquetas_predichas = set(df_eval["Etiqueta_Predicha"])
etiquetas_verdaderas = set(df_eval["Etiqueta_Verdadera"])
etiquetas_desconocidas = etiquetas_verdaderas - etiquetas_predichas

# Filtrar para el reporte solo etiquetas evaluables
df_eval_filtrado = df_eval[~df_eval["Etiqueta_Verdadera"].isin(etiquetas_desconocidas)].copy()

In [ ]:
# Crear reporte
reporte = classification_report(
    df_eval_filtrado["Etiqueta_Verdadera"],
    df_eval_filtrado["Etiqueta_Predicha"],
    output_dict=True,
    zero_division=0
)

In [ ]:
df_reporte = pd.DataFrame(reporte).transpose()
df_reporte

In [ ]:
etiquetas_desconocidas_sorted = sorted(etiquetas_desconocidas)
etiquetas_desconocidas_sorted

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
# Calcular la matriz de confusión
y_true = df_eval_filtrado["Etiqueta_Verdadera"]
y_pred = df_eval_filtrado["Etiqueta_Predicha"]
labels = sorted(y_true.unique())

cm = confusion_matrix(y_true, y_pred, labels=labels)

# Crear el heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel("Etiqueta Predicha")
plt.ylabel("Etiqueta Verdadera")
plt.title("Matriz de Confusión")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

# Calcular accuracy general
accuracy = accuracy_score(y_true, y_pred)
accuracy

## Prediccion de documentos

In [ ]:
#Crear embeddings con un modelo de Hugging Face
# Usamos un modelo multilingüe adecuado para textos legales
modelo = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Convertimos cada contenido a un vector (embedding)
corpus = df['Texto_Limpio'].tolist()
embeddings = modelo.encode(corpus, show_progress_bar=True)

# Convertimos a matriz numpy
embedding_matrix = np.array(embeddings)

In [ ]:
# Paso 4: Indexar embeddings con FAISS (búsqueda rápida por similitud)
dim = embedding_matrix.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embedding_matrix)

In [ ]:
def buscar_respuesta(pregunta, top_k=5, mostrar=True):
    """
    Busca los documentos más relevantes para una pregunta usando embeddings semánticos.

    Args:
        pregunta (str): pregunta del usuario
        top_k (int): cantidad de documentos a mostrar
        mostrar (bool): si se desea imprimir resultados o no

    Returns:
        df_resultado (pd.DataFrame): sub-DataFrame con los documentos más similares
        lista_indices (List[int]): lista de índices del DataFrame original
    """
    pregunta_emb = modelo.encode([pregunta])
    D, I = index.search(np.array(pregunta_emb), top_k)

    indices = I[0].tolist()
    distancias = D[0]
    resultados = df.iloc[indices].copy()
    resultados["Distancia"] = distancias

    if mostrar:
        for i, (idx, dist) in enumerate(zip(indices, distancias)):
            print(f"\n🔎 Documento {i+1} (Distancia {dist:.2f})")
            print(f"OP_Numero: {df.iloc[idx]['OP_Numero']}")
            print(f"Contenido:\n{df.iloc[idx]['Texto_Limpio'][:2000]}...")

    return resultados, indices


In [ ]:
# Paso 6: Probar el sistema
pregunta = "¿Qué documentos mencionan adjudicaciones?"
resultados = buscar_respuesta(pregunta)

In [ ]:
resultados[0]

In [ ]:
resultados[1]

In [ ]:
df.loc[df['OP_Numero']=="100114953"]['Texto_Limpio'].values[0]